In [0]:
import findspark
findspark.init('/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator


spark = SparkSession.builder.appName("analytics").getOrCreate()

# Support Vector Machines


## Ingestión de Datos:

Los datos se cargan en un DataFrame.

In [0]:
# Cargar el dataset
data = spark.read.csv("data/heart_failure_clinical_records_dataset.csv", header=True, inferSchema=True)



## Exploración de Datos:

In [0]:
data.printSchema()

In [0]:
# Mostramos las primeras filas del dataset
data.show(5)

## Selección de *features*


In [0]:
# Seleccionamos las características que vamos a utilizar y la variable a predecir
feature_columns = ["age", "anaemia", "creatinine_phosphokinase", "diabetes", "ejection_fraction", 
                   "high_blood_pressure", "platelets", "serum_creatinine", "serum_sodium", 
                   "sex", "smoking", "time"]
target_column = "DEATH_EVENT"


## VectorAssembler, Indexers, Scalers

Combina varias columnas de características en un solo vector de características, esencial para el entrenamiento del modelo.

In [0]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

indexer = StringIndexer(inputCol=target_column, outputCol="label")

## Modelo SVM


In [0]:
svm = LinearSVC(featuresCol="scaledFeatures", labelCol="label")

## Pipeline

In [0]:
# Crear un pipeline con los pasos anteriores
pipeline = Pipeline(stages=[assembler, scaler, indexer, svm])

## Entrenamiento

In [0]:
# Dividir los datos en conjuntos de entrenamiento (70%) y prueba (30%)
datos_entrenamiento, datos_prueba = data.randomSplit([0.7, 0.3], seed=42)

# Entrenar el modelo utilizando los datos de entrenamiento
modelo = pipeline.fit(datos_entrenamiento)


## Predicciones

In [0]:
# Realizar predicciones en los datos de prueba
predicciones = modelo.transform(datos_prueba)


## Revisión de resultados del modelo

In [0]:
# Evaluar el modelo utilizando un evaluador de clasificación binaria
evaluador = BinaryClassificationEvaluator(labelCol="label")
exactitud = evaluador.evaluate(predicciones)

print(f"Exactitud en el conjunto de prueba: {exactitud}")

Mostrar predicciones

In [0]:
# Mostrar las predicciones junto con algunas de las características originales
predicciones.select("age", "anaemia", "creatinine_phosphokinase", "diabetes", "ejection_fraction", 
                    "high_blood_pressure", "platelets", "serum_creatinine", "serum_sodium", 
                    "sex", "smoking", "time", "DEATH_EVENT", "prediction").show(5)